# SQL TWO
## Análise de Dados com SQL
### Aula 1 - DATE FORMAT, CTE, CASE WHEN, FILTER

### 1 - Diretório
#### Uma boa prática é inicialmente limpar o diretório, ou seja, excluir os arquivos (caso já tenham sido criados)

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/nossa_empresa

res0: Boolean = false

### 2 - Tabela nossa_empresa
#### Vamos criar uma tabela com dados fictícios para nossos tratamentos neste notebook. Utilizaremos DROP e CREATE TABLE para isso.

In [0]:
-- 1 - Criação da base de dados
DROP TABLE IF EXISTS NOSSA_EMPRESA;

CREATE TABLE NOSSA_EMPRESA
(
  id_funcionario INT,
  entrada_saida STRING,
  hora TIMESTAMP
)
USING DELTA;

### 3 - Inserindo dados de entrada e saída dos funcionários na tabela:
 - INSERT INTO
 - nome da tabela
    - (colunas da tabela)
 - VALUES
    - (dados de cada coluna separados por vírgula)

In [0]:
-- 2 - Inserindo dados na tabela
INSERT INTO
NOSSA_EMPRESA
(id_funcionario, entrada_saida, hora)
VALUES
(1, "entrada", "08:00:00"),
(1, "saída", "12:00:00"),
(1, "entrada", "13:00:00"),
(1, "saída", "17:00:00"),
(1, "entrada", "18:00:00"),

(2, "entrada", "09:00:00"),
(2, "saída", "11:30:00"),
(2, "entrada", "12:30:00"),
(2, "saída", "16:30:00"),
(2, "entrada", "19:00:00"),

(3, "entrada", "07:45:00"),
(3, "saída", "12:15:00"),
(4, "entrada", "10:00:00"),
(4, "saída", "14:00:00"),
(5, "entrada", "08:30:00"),
(5, "saída", "17:00:00");

num_affected_rows,num_inserted_rows
16,16


### 4 - Visualizando os dados

In [0]:
-- 3 - Visualizando os dados
SELECT * FROM NOSSA_EMPRESA;

id_funcionario,entrada_saida,hora
1,entrada,2025-12-19T08:00:00.000+0000
1,saída,2025-12-19T12:00:00.000+0000
1,entrada,2025-12-19T13:00:00.000+0000
1,saída,2025-12-19T17:00:00.000+0000
1,entrada,2025-12-19T18:00:00.000+0000
2,entrada,2025-12-19T09:00:00.000+0000
2,saída,2025-12-19T11:30:00.000+0000
2,entrada,2025-12-19T12:30:00.000+0000
2,saída,2025-12-19T16:30:00.000+0000
2,entrada,2025-12-19T19:00:00.000+0000


### 5 - Max, Group By
#### Verificar os maiores horários de saída por funcionário

In [0]:
-- 4 - Verificando o maior horário por empregado
SELECT
  id_funcionario,
  max(hora) 
FROM NOSSA_EMPRESA
GROUP BY id_funcionario;

id_funcionario,max(hora)
1,2025-12-19T18:00:00.000+0000
3,2025-12-19T12:15:00.000+0000
5,2025-12-19T17:00:00.000+0000
4,2025-12-19T14:00:00.000+0000
2,2025-12-19T19:00:00.000+0000


### 6 - DATE_FORMAT, MAX, MIN
#### Tratando a visualização da data com DATE FORMAT

In [0]:
-- 5 - Verificando o maior horário por funcionário com DATE_FORMAT e MAX
SELECT
  id_funcionario,
  date_format(max(hora), "HH:mm:ss") as horario_maximo 
FROM NOSSA_EMPRESA
GROUP BY id_funcionario
ORDER BY id_funcionario;

id_funcionario,horario_maximo
1,18:00:00
2,19:00:00
3,12:15:00
4,14:00:00
5,17:00:00


In [0]:
-- 6 - Verificando o menor horário por funcionário com DATE_FORMAT e MIN
SELECT  
  id_funcionario,
  DATE_FORMAT(MIN(hora), "HH:mm:ss") as horario_minimo
FROM NOSSA_EMPRESA
GROUP BY id_funcionario
ORDER BY id_funcionario;

id_funcionario,horario_minimo
1,08:00:00
2,09:00:00
3,07:45:00
4,10:00:00
5,08:30:00


### 7 - CASE WHEN
#### Podemos utilizar o CASE WHEN para verificar menor e maior horários registrado por funcionário. Se o horário de entrada estiver maior que o horário de saída, houve alguma irregularidade no ponto.

In [0]:
-- 7 - Verificando as maiores entradas e saídas case com CASE
SELECT
  id_funcionario,
  DATE_FORMAT(MAX(CASE WHEN entrada_saida = "entrada" THEN hora END), "HH:mm:ss") as horario_maximo_entrada,
  DATE_FORMAT(MIN(CASE WHEN entrada_saida = "saída" THEN hora END), "HH:mm:ss") as horario_minimo_saida
FROM NOSSA_EMPRESA
GROUP BY id_funcionario;

id_funcionario,horario_maximo_entrada,horario_minimo_saida
1,18:00:00,12:00:00
3,07:45:00,12:15:00
5,08:30:00,17:00:00
4,10:00:00,14:00:00
2,19:00:00,11:30:00


### 8 - CTE, CASE WHEN
#### É extremamente normal e produtivo utilizarmos tabelas temporárias, ou seja, existente somente durante a execução da consulta. No caso, o CTE é assim, permitindo maior flexibilidade ao SELECT externo, e para nossa análise, verificaremos a existência de irregularidades na marcação do ponto.

In [0]:
-- 8 - Verificando o resultado com CTE E CASE WHEN
WITH CTE AS
(
SELECT
  id_funcionario,
  DATE_FORMAT(MAX(CASE WHEN entrada_saida = "entrada" THEN hora END), "HH:mm:ss") as horario_maximo_entrada,
  DATE_FORMAT(MIN(CASE WHEN entrada_saida = "saída" THEN hora END), "HH:mm:ss") as horario_minimo_saida
FROM NOSSA_EMPRESA
GROUP BY id_funcionario
)

SELECT * FROM CTE
WHERE horario_maximo_entrada > horario_minimo_saida;

id_funcionario,horario_maximo_entrada,horario_minimo_saida
1,18:00:00,12:00:00
2,19:00:00,11:30:00


### 9 - CTE, FILTER
#### Outra opção para trazer o mesmo resultado é a utilização do FILTER.

In [0]:
-- 9 - CTE com FILTER
WITH CTE_FILTER
AS
(
SELECT
  id_funcionario,
  DATE_FORMAT(MAX(hora) FILTER(WHERE entrada_saida = "entrada"), "HH:mm:ss") as horario_maximo_entrada,
  DATE_FORMAT(MIN(hora) FILTER(WHERE entrada_saida = "saída"), "HH:mm:ss") as horario_minimo_saida
FROM NOSSA_EMPRESA
GROUP BY id_funcionario
)

SELECT * FROM CTE_FILTER
WHERE horario_maximo_entrada > horario_minimo_saida;

id_funcionario,horario_maximo_entrada,horario_minimo_saida
1,18:00:00,12:00:00
2,19:00:00,11:30:00
